# Question A

In [26]:
from gurobipy import Model, GRB, quicksum

# =======================
# Input Data
# =======================

# Classes (CRN) with demand and pattern type
# Patterns can be: 'TR', 'MW', 'MWF', 'Lab'
classes_data = {
    30522: {'demand': 41, 'pattern': 'TR'},
    30524: {'demand': 44, 'pattern': 'MW'},
    30529: {'demand': 44, 'pattern': 'TR'},
    30530: {'demand': 16, 'pattern': 'Lab'},
    30532: {'demand': 16, 'pattern': 'Lab'},
    30533: {'demand': 8,  'pattern': 'Lab'},
    34447: {'demand': 5,  'pattern': 'Lab'},
    42741: {'demand': 4,  'pattern': 'Lab'},
    30535: {'demand': 46, 'pattern': 'TR'},
    30536: {'demand': 39, 'pattern': 'TR'},
    32173: {'demand': 40, 'pattern': 'MWF'},
    30539: {'demand': 65, 'pattern': 'MW'},
    30541: {'demand': 70, 'pattern': 'TR'},
    30544: {'demand': 39, 'pattern': 'MW'},
    30548: {'demand': 15, 'pattern': 'Lab'},
    30550: {'demand': 13, 'pattern': 'Lab'},
    42308: {'demand': 19, 'pattern': 'Lab'},
    30556: {'demand': 45, 'pattern': 'MWF'},
    34608: {'demand': 31, 'pattern': 'TR'},
    43516: {'demand': 17, 'pattern': 'TR'},
    30561: {'demand': 20, 'pattern': 'TR'},
    41627: {'demand': 9,  'pattern': 'TR'},
    30562: {'demand': 10, 'pattern': 'MW'}
}

# Rooms and capacities
rooms_data = {
    '324': 10,
    '117': 55,
    '121': 25,
    '205': 20,
    '438': 25,
    'S0014': 20,
    '300': 100,
    '304': 100,
    '336': 100,
    '123': 120,
    '127': 130,
    '108': 80,
    'M0204': 70,
    'N0202': 80,
    'P0201': 50
}

# Timeslots by pattern
TR_times = [
    "TR_7:30-8:45", "TR_9:00-10:15", "TR_10:30-11:45", 
    "TR_12:00-1:15", "TR_1:30-2:45", "TR_3:00-4:15", "TR_4:30-5:45"
]
MWF_times = [
    "MWF_9:30-10:20", "MWF_10:30-11:20", "MWF_11:30-12:20"
]
MW_times = [
    "MW_1:30-2:45", "MW_3:00-4:15"
]
# Lab times: one day per week at 4:30-5:30PM
Lab_times = [
    "Lab_Mon_4:30-5:30", "Lab_Tue_4:30-5:30", "Lab_Wed_4:30-5:30", 
    "Lab_Thu_4:30-5:30", "Lab_Fri_4:30-5:30"
]

pattern_to_times = {
    'TR': TR_times,
    'MWF': MWF_times,
    'MW': MW_times,
    'Lab': Lab_times
}

C = list(classes_data.keys())
R = list(rooms_data.keys())

# =======================
# Model Initialization
# =======================
m = Model("ClassroomAssignment")

# =======================
# Decision Variables
# =======================
# x_{c,r,t} = 1 if class c is assigned to room r and timeslot t
x = {}
for c in C:
    pat = classes_data[c]['pattern']
    demand = classes_data[c]['demand']
    for t in pattern_to_times[pat]:
        for r in R:
            # Only create var if capacity is sufficient
            if demand <= rooms_data[r]:
                x[(c,r,t)] = m.addVar(vtype=GRB.BINARY, name=f"x_{c}_{r}_{t}")

# y_r = 1 if room r is used
y = {r: m.addVar(vtype=GRB.BINARY, name=f"y_{r}") for r in R}

m.update()

# =======================
# Objective
# =======================
# Minimize the number of rooms used
m.setObjective(quicksum(y[r] for r in R), GRB.MINIMIZE)

# =======================
# Constraints
# =======================

# 1. Each class must be assigned exactly once
for c in C:
    pat = classes_data[c]['pattern']
    # sum over all rooms and all valid timeslots for that class
    m.addConstr(quicksum(x[(c,r,t)] for t in pattern_to_times[pat] for r in R if (c,r,t) in x) == 1, 
                name=f"ClassAssigned_{c}")

# 2. Capacity 
for (c,r,t) in x:
    m.addConstr(x[(c,r,t)] * classes_data[c]['demand'] <= rooms_data[r], f"Cap_{c}_{r}_{t}")

# 3. No two classes in the same room and same timeslot
# For each room and timeslot, sum of x must be <= 1
# Collect all (c,r,t) by room and timeslot
room_times = {}
for (c,r,t) in x.keys():
    room_times.setdefault((r,t), []).append((c,r,t))

for (r,t), assignments in room_times.items():
    m.addConstr(quicksum(x[a] for a in assignments) <= 1, name=f"NoConflict_{r}_{t}")

# 4. Linking room usage y_r
for (c,r,t) in x:
    m.addConstr(x[(c,r,t)] <= y[r], name=f"Link_{c}_{r}_{t}")

# =======================
# Solve the model
# =======================
m.optimize()

# =======================
# Print solution
# =======================
status = m.Status
if status == GRB.OPTIMAL or status == GRB.FEASIBLE:
    print("Solution found:")
    # Print which classes are assigned to which room and timeslot
    for c in C:
        for r in R:
            pat = classes_data[c]['pattern']
            for t in pattern_to_times[pat]:
                if (c,r,t) in x and x[(c,r,t)].X > 0.5:
                    print(f"Class {c} assigned to Room {r} at {t}, Demand={classes_data[c]['demand']}, Capacity={rooms_data[r]}")
    print("\nRooms used:")
    for r in R:
        if y[r].X > 0.5:
            print(f"Room {r} is used.")
else:
    print("No feasible solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3119 rows, 1443 columns and 7140 nonzeros
Model fingerprint: 0xf64b0e0c
Variable types: 0 continuous, 1443 integer (1443 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 13.0000000
Presolve removed 2867 rows and 11 columns
Presolve time: 0.01s
Presolved: 252 rows, 1432 columns, 3063 nonzeros
Variable types: 0 continuous, 1432 integer (1432 binary)
Found heuristic solution: objective 12.0000000

Root relaxation: objective 2.000000e+00, 368 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H  

# Question B

# model 1

In [43]:
import gurobipy as gp
from gurobipy import GRB
import re
import itertools

# -----------------------------
# INPUT DATA
# -----------------------------
courses = {
    'Owen Tyler Dodd': [
        {'CRN': 30522, 'days': ['T', 'R'], 'demand': 41}
    ],
    'Rui Zhu': [
        {'CRN': 30524, 'days': ['M', 'W'], 'demand': 44}
    ],
    'Talayeh Razzaghi': [
        {'CRN': 43516, 'days': ['T', 'R'], 'demand': 17}
    ],
    'Theodore B Trafalis': [
        {'CRN': 41627, 'days': ['T', 'R'], 'demand': 9}
    ],
    'Shivakumar Raman': [
        {'CRN': 30529, 'days': ['T', 'R'], 'demand': 44},
        {'CRN': 30530, 'days': 'Lab', 'demand': 16},
        {'CRN': 30532, 'days': 'Lab', 'demand': 16},
        {'CRN': 30533, 'days': 'Lab', 'demand': 8},
        {'CRN': 34447, 'days': 'Lab', 'demand': 5},
        {'CRN': 42741, 'days': 'Lab', 'demand': 4}
    ],
    'Charles D Nicholson': [
        {'CRN': 30535, 'days': ['T', 'R'], 'demand': 46},
        {'CRN': 34608, 'days': ['T', 'R'], 'demand': 31}
    ],
    'Cesar Alexander Ruiz Torres': [
        {'CRN': 30536, 'days': ['T', 'R'], 'demand': 39}
    ],
    'Kash A Barker': [
        {'CRN': 32173, 'days': ['M', 'W', 'F'], 'demand': 40},
        {'CRN': 30539, 'days': ['M', 'W'], 'demand': 65}
    ],
    'Andres D Gonzalez': [
        {'CRN': 30541, 'days': ['T', 'R'], 'demand': 70},
        {'CRN': 30561, 'days': ['T', 'R'], 'demand': 20}
    ],
    'Ziho Kang': [
        {'CRN': 30544, 'days': ['M', 'W'], 'demand': 39},
        {'CRN': 30548, 'days': 'Lab', 'demand': 15},
        {'CRN': 30550, 'days': 'Lab', 'demand': 13},
        {'CRN': 42308, 'days': 'Lab', 'demand': 19},
        {'CRN': 30556, 'days': ['M', 'W', 'F'], 'demand': 45},
        {'CRN': 30562, 'days': ['M', 'W'], 'demand': 10}
    ]
}

rooms = {
    '324': {'capacity': 10, 'building': 'BL'},
    '117': {'capacity': 55, 'building': 'CEC'},
    '121': {'capacity': 25, 'building': 'CEC'},
    '205': {'capacity': 20, 'building': 'CEC'},
    '438': {'capacity': 25, 'building': 'CEC'},
    'S0014': {'capacity': 20, 'building': 'CEC'},
    '300': {'capacity': 100, 'building': 'FH'},
    '304': {'capacity': 100, 'building': 'FH'},
    '336': {'capacity': 100, 'building': 'FH'},
    '123': {'capacity': 120, 'building': 'GLCH'},
    '127': {'capacity': 130, 'building': 'GLG'},
    '108': {'capacity': 80, 'building': 'PHSC'},
    'M0204': {'capacity': 70, 'building': 'SEC'},
    'N0202': {'capacity': 80, 'building': 'SEC'},
    'P0201': {'capacity': 50, 'building': 'SEC'}
}


def get_pattern(days):
    if days == 'Lab':
        return 'Lab'
    dset = set(days)
    if dset == {'T','R'}:
        return 'TR'
    elif dset == {'M','W'}:
        return 'MW'
    elif dset == {'M','W','F'}:
        return 'MWF'
    else:
        return 'Lab'

AllDays = ['M','T','W','R','F']

time_slots = {
    'M': {0: '9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'T': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'W': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'R': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'F': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM'}
}

# Add distinct lab slot for each day
time_slots['M'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['T'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['W'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['R'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['F'][3] = '4:30 PM - 5:30 PM (Lab)'

lab_slot = {'M':5,'T':7,'W':5,'R':7,'F':3}


def get_pattern(days):
    if days == 'Lab':
        return 'Lab'
    dset = set(days)
    if dset == {'T','R'}:
        return 'TR'
    elif dset == {'M','W'}:
        return 'MW'
    elif dset == {'M','W','F'}:
        return 'MWF'
    else:
        return 'Lab'

AllDays = ['M','T','W','R','F']

time_slots = {
    'M': {0: '9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'T': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'W': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM',3:'1:30 PM - 2:45 PM',4:'3:00 PM - 4:15 PM'},
    'R': {0:'7:30 AM - 8:45 AM',1:'9:00 AM - 10:15 AM',2:'10:30 AM - 11:45 AM',3:'12:00 PM - 1:15 PM',4:'1:30 PM - 2:45 PM',5:'3:00 PM - 4:15 PM',6:'4:30 PM - 5:45 PM'},
    'F': {0:'9:30 AM - 10:20 AM',1:'10:30 AM - 11:20 AM',2:'11:30 AM - 12:20 PM'}
}

# Add distinct lab slot for each day
time_slots['M'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['T'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['W'][5] = '4:30 PM - 5:30 PM (Lab)'
time_slots['R'][7] = '4:30 PM - 5:30 PM (Lab)'
time_slots['F'][3] = '4:30 PM - 5:30 PM (Lab)'

lab_slot = {'M':5,'T':7,'W':5,'R':7,'F':3}

Classes = []
for instr,clist in courses.items():
    clean_instr = instr.strip()
    for c in clist:
        CRN = c['CRN']
        days = c['days']
        dem = c['demand']
        pattern = get_pattern(days)
        DD = tuple(days) if pattern!='Lab' else tuple(AllDays)
        Classes.append((CRN, clean_instr, pattern, DD, dem))

ClassInfo = {c[0]: c for c in Classes}
FeasibleRooms = {CRN: [r for r in rooms if rooms[r]['capacity']>=ClassInfo[CRN][4]] for CRN in ClassInfo}

# FeasibleDayTime calculation:
FeasibleDayTime = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    feasible = []
    if P=='Lab':
        # Only lab slots each day
        for d in AllDays:
            feasible.append((d, lab_slot[d], P))
    else:
        for d in DD:
            for t_idx in time_slots[d].keys():
                if t_idx != lab_slot[d]:
                    feasible.append((d,t_idx,P))
    FeasibleDayTime[CRN] = feasible

OfficeBuilding = "CEC"
W_intra = 7
W_inter = 12

def building_of_room(r):
    return rooms[r]['building']

def cost_between_buildings(b1, b2, OfficeBuilding="CEC", W_intra=7, W_inter=12):
    if b1 == b2:
        return W_intra
    if b1==OfficeBuilding or b2==OfficeBuilding:
        if b1!=b2:
            return W_inter
        else:
            return W_intra
    return W_inter

# Define consecutive: now only if t2 == t1+1
Consecutive = {}
for d in time_slots:
    sorted_ts = sorted(time_slots[d].keys())
    for t1 in sorted_ts:
        for t2 in sorted_ts:
            Consecutive[(d,t1,t2)] = 1 if t2 == t1+1 else 0

Instructors = set([ClassInfo[c][1] for c in ClassInfo])
InstructorClasses = {i:[CRN for CRN,(C,I2,P,DD,dem) in ClassInfo.items() if I2==i] for i in Instructors}

BuildingsSet = set(building_of_room(r) for r in rooms)

m = gp.Model("Minimize_Actual_Walking")

x = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    for (d,t,pp) in FeasibleDayTime[CRN]:
        for r in FeasibleRooms[CRN]:
            x[(CRN,r,d,t)] = m.addVar(vtype=GRB.BINARY, name=f"x_{CRN}_{r}_{d}_{t}")

Z = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        Z[(CRN,d)] = m.addVar(vtype=GRB.BINARY, name=f"Z_{CRN}_{d}")

y = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1 in c_i:
            for c2 in c_i:
                if c1!=c2:
                    y[(c1,c2,d)] = m.addVar(vtype=GRB.BINARY, name=f"y_{c1}_{c2}_{d}")

b_ind = {}
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        for bldg in BuildingsSet:
            b_ind[(CRN,d,bldg)] = m.addVar(vtype=GRB.BINARY, name=f"b_ind_{CRN}_{d}_{bldg}")

w_var = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            for r1 in FeasibleRooms[c1]:
                for r2 in FeasibleRooms[c2]:
                    w_var[(c1,c2,d,r1,r2)] = m.addVar(vtype=GRB.BINARY, name=f"w_{c1}_{c2}_{d}_{r1}_{r2}")

f = {}
l = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c in c_i:
            f[(c,d)] = m.addVar(vtype=GRB.BINARY, name=f"f_{c}_{d}")
            l[(c,d)] = m.addVar(vtype=GRB.BINARY, name=f"l_{c}_{d}")

break_var = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                break_var[(c1,c2,d)] = m.addVar(vtype=GRB.BINARY, name=f"break_{c1}_{c2}_{d}")


m.update()

# Capacity
for (CRN,r,d,t) in x:
    dem = ClassInfo[CRN][4]
    cap = rooms[r]['capacity']
    m.addConstr(x[(CRN,r,d,t)] <= 1*(cap>=dem))

# Assignment constraints
for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    if P=='Lab':
        lhs = gp.quicksum(x[(CRN,r,d,t)] for (d,t,pp) in FeasibleDayTime[CRN] for r in FeasibleRooms[CRN])
        m.addConstr(lhs == 1)
    else:
        for dd in DD:
            lhs = gp.quicksum(x[(CRN,r,dd,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==dd for r in FeasibleRooms[CRN])
            m.addConstr(lhs == 1)

# Pattern consistency
def pattern_days(P):
    if P=='TR':
        return ['T','R']
    elif P=='MW':
        return ['M','W']
    elif P=='MWF':
        return ['M','W','F']
    else:
        return None

for CRN,(C,I,P,DD,dem) in ClassInfo.items():
    if P in ['TR','MW','MWF']:
        p_days = pattern_days(P)
        day_times = {d:[t for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d] for d in p_days}
        if P=='TR':
            for t_T in day_times[p_days[0]]:
                if t_T in day_times[p_days[1]]:
                    lhs_day1 = gp.quicksum(x[(CRN,r,p_days[0],t_T)] for r in FeasibleRooms[CRN])
                    lhs_day2 = gp.quicksum(x[(CRN,r,p_days[1],t_T)] for r in FeasibleRooms[CRN])
                    m.addConstr(lhs_day1 == lhs_day2)
        elif P=='MW':
            for t_M in day_times[p_days[0]]:
                if t_M in day_times[p_days[1]]:
                    lhs_M = gp.quicksum(x[(CRN,r,'M',t_M)] for r in FeasibleRooms[CRN])
                    lhs_W = gp.quicksum(x[(CRN,r,'W',t_M)] for r in FeasibleRooms[CRN])
                    m.addConstr(lhs_M == lhs_W)
        elif P=='MWF':
            common_ts = set(day_times['M']).intersection(day_times['W'], day_times['F'])
            for t_m in common_ts:
                lhs_M = gp.quicksum(x[(CRN,r,'M',t_m)] for r in FeasibleRooms[CRN])
                lhs_W = gp.quicksum(x[(CRN,r,'W',t_m)] for r in FeasibleRooms[CRN])
                lhs_F = gp.quicksum(x[(CRN,r,'F',t_m)] for r in FeasibleRooms[CRN])
                m.addConstr(lhs_M == lhs_W)
                m.addConstr(lhs_W == lhs_F)

# Room availability
for d in AllDays:
    for t in time_slots[d]:
        for rr in rooms:
            m.addConstr(gp.quicksum(x[(CRN,rr,d,t)] for CRN in ClassInfo if (CRN,rr,d,t) in x) <= 1)

# Instructor non-overlap
'''for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for t in time_slots[d]:
            m.addConstr(gp.quicksum(x[(CRN,r,d,t)] for CRN in c_i for r in FeasibleRooms[CRN] if (CRN,r,d,t) in x) <= 1)'''

# Overlapping Time Slots: Explicit Mapping
overlapping_slots = {}
for d in time_slots:
    overlapping_slots[d] = []
    for t1, t2 in itertools.combinations(time_slots[d].keys(), 2):
        start_1, end_1 = map(int, re.findall(r'\d+', time_slots[d][t1].replace(":", "").replace(" AM", "").replace(" PM", "")))
        start_2, end_2 = map(int, re.findall(r'\d+', time_slots[d][t2].replace(":", "").replace(" AM", "").replace(" PM", "")))
        if start_1 < end_2 and start_2 < end_1:  # Overlap condition
            overlapping_slots[d].append((t1, t2))

# Instructor non-overlap with overlapping time slots
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        # Prevent assignments to same slot
        for t in time_slots[d]:
            m.addConstr(
                gp.quicksum(
                    x[(CRN, r, d, t)] for CRN in c_i for r in FeasibleRooms[CRN] if (CRN, r, d, t) in x
                ) <= 1,
                f"Instructor_{i}_NonOverlap_{d}_{t}"
            )
        
        # Prevent assignments to overlapping slots
        for t1, t2 in overlapping_slots[d]:
            m.addConstr(
                gp.quicksum(
                    x[(CRN, r, d, t)] for CRN in c_i for r in FeasibleRooms[CRN] for t in [t1, t2]
                    if (CRN, r, d, t) in x
                ) <= 1,
                f"Instructor_{i}_Overlap_{d}_{t1}_{t2}"
            )

# Z linking
for (CRN,(C,I,P,DD,dem)) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        lhs = gp.quicksum(x[(CRN,r,d,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d for r in FeasibleRooms[CRN])
        m.addConstr(Z[(CRN,d)] == lhs)

# Building indicators
for (CRN,(C,I,P,DD,dem)) in ClassInfo.items():
    valid_days = DD if P!='Lab' else AllDays
    for d in valid_days:
        m.addConstr(gp.quicksum(b_ind[(CRN,d,b)] for b in BuildingsSet) == Z[(CRN,d)])
        for b in BuildingsSet:
            lhs = gp.quicksum(x[(CRN,r,d,t)] for (d2,t,pp) in FeasibleDayTime[CRN] if d2==d for r in FeasibleRooms[CRN] if building_of_room(r)==b)
            m.addConstr(b_ind[(CRN,d,b)] == lhs)

# A[c,d,t]
A = {}
for CRN in ClassInfo:
    for (d,t,pp) in FeasibleDayTime[CRN]:
        A[(CRN,d,t)] = gp.quicksum(x[(CRN,r,d,t)] for r in FeasibleRooms[CRN])

# Consecutive logic: 
# We've already defined y and A. We must link y to consecutive timeslots.
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                Ytt_list = []
                feasible_c1 = [(d2,t2) for (d2,t2,pp) in FeasibleDayTime[c1] if d2==d]
                feasible_c2 = [(d3,t3) for (d3,t3,pp) in FeasibleDayTime[c2] if d3==d]
                c1_ts = [t1 for (dd,t1) in feasible_c1]
                c2_ts = [t2 for (dd,t2) in feasible_c2]

                for t1 in c1_ts:
                    for t2 in c2_ts:
                        if (d,t1,t2) in Consecutive:
                            # Consecutive=1 if t2 == t1+1 else 0
                            if Consecutive[(d,t1,t2)]==1:
                                Ytt_var = m.addVar(vtype=GRB.BINARY, name=f"Ytt_{c1}_{c2}_{d}_{t1}_{t2}")
                                m.addConstr(Ytt_var <= A[(c1,d,t1)])
                                m.addConstr(Ytt_var <= A[(c2,d,t2)])
                                # If consecutive=1, we need Ytt_var ≥ A[c1,d,t1]+A[c2,d,t2]-1 (both must be assigned)
                                m.addConstr(Ytt_var >= A[(c1,d,t1)]+A[(c2,d,t2)]-1)
                                Ytt_list.append(Ytt_var)
                # y ≤ sum(Ytt_list)
                m.addConstr(y[(c1,c2,d)] <= gp.quicksum(Ytt_list))
                # If no consecutive pairs chosen, y=0
                # If at least one consecutive pair chosen, y≥1. For simplicity:
                m.addConstr(gp.quicksum(Ytt_list) >= y[(c1,c2,d)])


# f and l constraints
for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:
            m.addConstr(f[(c,d)] <= Z[(c,d)])
            m.addConstr(f[(c,d)] + gp.quicksum(y[(c2,c,d)] for c2 in c_i if c2!=c) <= 1)
            m.addConstr(l[(c,d)] <= Z[(c,d)])
            m.addConstr(l[(c,d)] + gp.quicksum(y[(c,c2,d)] for c2 in c_i if c2!=c) <= 1)

    for d in AllDays:
        m.addConstr(gp.quicksum(f[(c,d)] for c in c_i if (c,d) in f) <= 1)
        m.addConstr(gp.quicksum(l[(c,d)] for c in c_i if (c,d) in l) <= 1)


# w_var linking:
for (c1,c2,d,r1,r2) in w_var:
    lhs_c1 = gp.quicksum(x[(c1,r1,d,t)] for (d2,t,pp) in FeasibleDayTime[c1] if d2==d)
    lhs_c2 = gp.quicksum(x[(c2,r2,d,t)] for (d2,t,pp) in FeasibleDayTime[c2] if d2==d)
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= lhs_c1)
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= lhs_c2)

# break_var:
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                zc1 = Z[(c1,d)] if (c1,d) in Z else 0
                zc2 = Z[(c2,d)] if (c2,d) in Z else 0
                #m.addConstr(break_var[(c1,c2,d)] <= (zc1 if zc1!=0 else 0))
                # Ensure `break_var` only defined if both `c1` and `c2` have classes on day `d`
                if (c1, d) in Z and (c2, d) in Z:
                    m.addConstr(break_var[(c1, c2, d)] <= Z[(c1, d)])
                    m.addConstr(break_var[(c1, c2, d)] <= Z[(c2, d)])
                    m.addConstr(break_var[(c1, c2, d)] <= 2 - (y[(c1, c2, d)] + y[(c2, c1, d)]))
                    m.addConstr(
                        break_var[(c1, c2, d)] >= Z[(c1, d)] + Z[(c2, d)] - (y[(c1, c2, d)] + y[(c2, c1, d)]) - 1
                    )
                else:
                    m.addConstr(break_var[(c1, c2, d)] == 0)
brw = {}
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        brw[(c1,c2,d,b1,b2)] = m.addVar(vtype=GRB.BINARY, name=f"brw_{c1}_{c2}_{d}_{b1}_{b2}")


m.update()

for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        if (c1,d,b1) in b_ind and (c2,d,b2) in b_ind:
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= break_var[(c1,c2,d)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= b_ind[(c1,d,b1)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] <= b_ind[(c2,d,b2)])
                            m.addConstr(brw[(c1,c2,d,b1,b2)] >= b_ind[(c1,d,b1)]+b_ind[(c2,d,b2)]+break_var[(c1,c2,d)]-2)
                        else:
                            m.addConstr(brw[(c1,c2,d,b1,b2)] == 0)

for (c1,c2,d,r1,r2) in w_var:
    m.addConstr(w_var[(c1,c2,d,r1,r2)] <= y[(c1,c2,d)])


obj_expr = gp.LinExpr()

# Office->first class and last class->office costs, break costs, consecutive costs:
# For simplicity, we keep same logic as before.

for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        # f and building:
        for d in valid_days:
            for b in BuildingsSet:
                fh = m.addVar(vtype=GRB.BINARY, name=f"fh_{c}_{d}_{b}")
                m.addConstr(fh <= f[(c,d)])
                m.addConstr(fh <= b_ind[(c,d,b)])
                m.addConstr(fh >= f[(c,d)]+b_ind[(c,d,b)]-1)
                office_to_c_cost = cost_between_buildings(OfficeBuilding,b,OfficeBuilding,W_intra,W_inter)
                obj_expr += office_to_c_cost * fh

# Consecutive classes cost:
for (c1,c2,d,r1,r2) in w_var:
    b1 = building_of_room(r1)
    b2 = building_of_room(r2)
    trans_cost = cost_between_buildings(b1,b2,OfficeBuilding,W_intra,W_inter)
    obj_expr += trans_cost * w_var[(c1, c2, d, r1, r2)]

# break cost:
for i in Instructors:
    c_i = InstructorClasses[i]
    for d in AllDays:
        for c1,c2 in itertools.permutations(c_i,2):
            if c1!=c2:
                for b1 in BuildingsSet:
                    for b2 in BuildingsSet:
                        return_office_cost = cost_between_buildings(b1,OfficeBuilding,OfficeBuilding,W_intra,W_inter) \
                                             + cost_between_buildings(OfficeBuilding,b2,OfficeBuilding,W_intra,W_inter)
                        obj_expr += return_office_cost * brw[(c1, c2, d, b1, b2)]


# Last class->office:
for i in Instructors:
    c_i = InstructorClasses[i]
    for c in c_i:
        (CRN,I,P,DD,dem) = ClassInfo[c]
        valid_days = DD if P!='Lab' else AllDays
        for d in valid_days:
            for b in BuildingsSet:
                lh = m.addVar(vtype=GRB.BINARY, name=f"lh_{c}_{d}_{b}")
                m.addConstr(lh <= l[(c,d)])
                m.addConstr(lh <= b_ind[(c,d,b)])
                m.addConstr(lh >= l[(c,d)]+b_ind[(c,d,b)]-1)
                off_cost = cost_between_buildings(b,OfficeBuilding,OfficeBuilding,W_intra,W_inter)
                obj_expr += off_cost * lh



m.setObjective(obj_expr, GRB.MINIMIZE)

m.optimize()


if m.status == GRB.OPTIMAL:
    print("Optimal solution found with objective:", m.objVal)
    
    # Extract chosen assignments
    chosen = []
    for (CRN,r,d,t) in x:
        if x[(CRN,r,d,t)].X > 0.5:
            (C,I,P,DD,dem) = ClassInfo[CRN]
            chosen.append((I, d, t, CRN, r, P))
    
    def slot_start_minutes(slot_str):
        match = re.match(r'(\d+):(\d+) (\wM)', slot_str)
        hour = int(match.group(1))
        minute = int(match.group(2))
        ampm = match.group(3)
        if ampm == 'PM' and hour < 12:
            hour += 12
        return hour*60 + minute
    
    schedule = {}
    for (I,d,t,CRN,r,P) in chosen:
        slot_str = time_slots[d][t]
        start_min = slot_start_minutes(slot_str)
        bldg = building_of_room(r)
        if I not in schedule:
            schedule[I] = {}
        if d not in schedule[I]:
            schedule[I][d] = []
        schedule[I][d].append((start_min, CRN, r, P, slot_str, bldg))
    
    # Sort each day by start time
    for I in schedule:
        for d in schedule[I]:
            schedule[I][d].sort(key=lambda x: x[0])
    
    # Print the schedule
    for I in sorted(schedule.keys()):
        print(f"\nSchedule for Instructor: {I}")
        for d in ['M','T','W','R','F']:
            if d in schedule[I]:
                print(f"  {d}:")
                for (start_min, CRN, r, P, slot_str, bldg) in schedule[I][d]:
                    print(f"    {slot_str} -> CRN {CRN} ({P}), Room {r}, Building {bldg}")
else:
    print("No optimal solution found. Status:", m.status)



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 226013 rows, 77576 columns and 536107 nonzeros
Model fingerprint: 0x4bf9e928
Variable types: 0 continuous, 77576 integer (77576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 192604 rows and 66023 columns
Presolve time: 0.18s
Presolved: 33409 rows, 11553 columns, 98008 nonzeros
Variable types: 0 continuous, 11553 integer (11553 binary)

Root relaxation: objective 0.000000e+00, 664 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   39          -    0.00000      -     -    0s
H   

In [37]:
if m.status == GRB.OPTIMAL:
    print("Optimal solution found with objective:", m.objVal)
    
    total_cost = 0
    first_last_cost = 0
    consecutive_cost = 0
    break_cost = 0
    
    # First/Last Class Costs
    print("\nCost for First/Last Classes:")
    for i in Instructors:
        c_i = InstructorClasses[i]
        for d in AllDays:
            # First Class
            for c in c_i:
                for b in BuildingsSet:
                    if f[(c, d)].X > 0.5 and b_ind[(c, d, b)].X > 0.5:
                        cost_to_office = cost_between_buildings(OfficeBuilding, b, OfficeBuilding, W_intra, W_inter)
                        print(f"Instructor {i}, Day {d}, First Class in Building {b}: Cost {cost_to_office}")
                        first_last_cost += cost_to_office
            
            # Last Class
            for c in c_i:
                for b in BuildingsSet:
                    if l[(c, d)].X > 0.5 and b_ind[(c, d, b)].X > 0.5:
                        cost_from_office = cost_between_buildings(b, OfficeBuilding, OfficeBuilding, W_intra, W_inter)
                        print(f"Instructor {i}, Day {d}, Last Class in Building {b}: Cost {cost_from_office}")
                        first_last_cost += cost_from_office
    
    # Consecutive Class Costs
    print("\nCost for Consecutive Classes:")
    for (c1, c2, d, r1, r2) in w_var:
        if w_var[(c1, c2, d, r1, r2)].X > 0.5:
            b1 = building_of_room(r1)
            b2 = building_of_room(r2)
            cost_between_classes = cost_between_buildings(b1, b2, OfficeBuilding, W_intra, W_inter)
            print(f"Instructor Transition from {c1} in {b1} to {c2} in {b2} on Day {d}: Cost {cost_between_classes}")
            consecutive_cost += cost_between_classes
    
    # Break Costs
    print("\nCost for Breaks:")
    for i in Instructors:
        c_i = InstructorClasses[i]
        for d in AllDays:
            for c1, c2 in itertools.permutations(c_i, 2):
                if break_var[(c1, c2, d)].X > 0.5:
                    for b1 in BuildingsSet:
                        for b2 in BuildingsSet:
                            if brw[(c1, c2, d, b1, b2)].X > 0.5:
                                return_to_office_cost = cost_between_buildings(b1, OfficeBuilding, OfficeBuilding, W_intra, W_inter) \
                                                        + cost_between_buildings(OfficeBuilding, b2, OfficeBuilding, W_intra, W_inter)
                                print(f"Instructor {i}, Break from {c1} in {b1} to {c2} in {b2} on Day {d}: Cost {return_to_office_cost}")
                                break_cost += return_to_office_cost
    
    total_cost = first_last_cost + consecutive_cost + break_cost
    print("\n--- Summary of Costs ---")
    print(f"First/Last Class Cost: {first_last_cost}")
    print(f"Consecutive Class Cost: {consecutive_cost}")
    print(f"Break Cost: {break_cost}")
    print(f"Total Calculated Cost: {total_cost}")
    

Optimal solution found with objective: 112.0

Cost for First/Last Classes:

Cost for Consecutive Classes:

Cost for Breaks:
Instructor Shivakumar Raman, Break from 30529 in CEC to 30530 in CEC on Day T: Cost 14
Instructor Shivakumar Raman, Break from 30530 in CEC to 30529 in CEC on Day T: Cost 14
Instructor Shivakumar Raman, Break from 30529 in CEC to 34447 in CEC on Day R: Cost 14
Instructor Shivakumar Raman, Break from 34447 in CEC to 30529 in CEC on Day R: Cost 14
Instructor Ziho Kang, Break from 30556 in CEC to 30562 in CEC on Day M: Cost 14
Instructor Ziho Kang, Break from 30562 in CEC to 30556 in CEC on Day M: Cost 14
Instructor Ziho Kang, Break from 30556 in CEC to 30562 in CEC on Day W: Cost 14
Instructor Ziho Kang, Break from 30562 in CEC to 30556 in CEC on Day W: Cost 14

--- Summary of Costs ---
First/Last Class Cost: 0
Consecutive Class Cost: 0
Break Cost: 112
Total Calculated Cost: 112
